In [19]:
# 600519.XSHG 贵州茅台
# 000858.XSHE 五粮液

# 000001.XSHE 平安银行
# 600036.XSHG 招商银行

# 600104.XSHG 上汽集团
# 600066.XSHG 宇通客车

# 600085.XSHG 同仁堂
# 600276.XSHG 恒瑞医药

# 600895.XSHG 张江高科
# 600376.XSHG 首开股份

In [20]:
class Para:

   index_code = '600066.XSHG' # eg. '000300.XSHG' 'CS' 600519.XSHG
   stock_name = 'yt'
   start_date = '2012-01-01'
   end_date = '2021-12-31' # 2022-01-01
   interval = 1
   yield_rate_interval = 20

   write_folder_path = 'data/'+stock_name+'_space_'+str(interval)+'d_rate_'+str(yield_rate_interval)+'d_'+str(start_date)[2:4]+'-'+str(end_date)[2:4]+'_raw'

   # 估值
   valuation_factor = [
      'ep_ratio_ttm',
      'pb_ratio_ttm',
      'sp_ratio_ttm']

   # 技术
   technology_factor = [
      'MACD_DIFF',
      'MACD_DEA',
      'MACD_HIST',
      'RSI10',
      'SY',
      'BIAS20']

   # 其他
   other_factor = [
      'VOL30',# 换手率
      'VOL60',
      'VOL120',
      'VOLT20',# 波动率
      'VOLT60',
      'AR',# 人气意愿指标
      'BR']

   factor_list = valuation_factor + technology_factor + other_factor

para = Para()
print(para.write_folder_path)

data/yt_space_1d_rate_20d_12-21_raw


In [21]:
import rqdatac

rqdatac.init()

c:\users\millet\miniconda3\envs\torch1.8-gpu\lib\site-packages\rqdatac\client.py:197: UserWarning: rqdatac is already inited. Settings will be changed.
  warnings.warn("rqdatac is already inited. Settings will be changed.", stacklevel=0)


In [22]:
from rqdatac import get_trading_dates

trading_dates_list = get_trading_dates(start_date=para.start_date, end_date=para.end_date)
trading_dates_list_interval = trading_dates_list[0: len(trading_dates_list) - para.yield_rate_interval: para.interval]
trading_dates_list_interval

[datetime.date(2012, 1, 4),
 datetime.date(2012, 1, 5),
 datetime.date(2012, 1, 6),
 datetime.date(2012, 1, 9),
 datetime.date(2012, 1, 10),
 datetime.date(2012, 1, 11),
 datetime.date(2012, 1, 12),
 datetime.date(2012, 1, 13),
 datetime.date(2012, 1, 16),
 datetime.date(2012, 1, 17),
 datetime.date(2012, 1, 18),
 datetime.date(2012, 1, 19),
 datetime.date(2012, 1, 20),
 datetime.date(2012, 1, 30),
 datetime.date(2012, 1, 31),
 datetime.date(2012, 2, 1),
 datetime.date(2012, 2, 2),
 datetime.date(2012, 2, 3),
 datetime.date(2012, 2, 6),
 datetime.date(2012, 2, 7),
 datetime.date(2012, 2, 8),
 datetime.date(2012, 2, 9),
 datetime.date(2012, 2, 10),
 datetime.date(2012, 2, 13),
 datetime.date(2012, 2, 14),
 datetime.date(2012, 2, 15),
 datetime.date(2012, 2, 16),
 datetime.date(2012, 2, 17),
 datetime.date(2012, 2, 20),
 datetime.date(2012, 2, 21),
 datetime.date(2012, 2, 22),
 datetime.date(2012, 2, 23),
 datetime.date(2012, 2, 24),
 datetime.date(2012, 2, 27),
 datetime.date(2012, 2, 2

In [23]:
len(trading_dates_list_interval)

2411

In [24]:
from my_utils.data_process import remove_st
from rqdatac import index_components, get_factor, get_price, is_suspended, get_next_trading_date, instruments, all_instruments
import time
import datetime
import numpy as np
import pandas as pd

time_begin = time.time()
for j in range(len(trading_dates_list_interval)):

    print()
    print("number of trading_dates_list_interval: {}/{}".format(j, len(trading_dates_list_interval)-1))
    print("use time: {}".format(time.time()-time_begin))

    date = trading_dates_list_interval[j]

    # 全部股票
    if para.index_code == 'CS':
        stocks_list = all_instruments(type=para.index_code, date=date)['order_book_id'].values.tolist()
    # hs300
    elif para.index_code == '000300.XSHG':
        stocks_list = index_components(order_book_id=para.index_code, date=date)
    # 个股
    else:
        stocks_list = [para.index_code]

    # 剔除 ST 股票
    stocks_list_no_st = remove_st(stocks_list=stocks_list, date=date)
    # 获取因子
    df = get_factor(order_book_ids=stocks_list_no_st, factor=para.factor_list, date=date)
    # 处理多级索引
    df = df.reset_index(level='date')
    # 插入 month 列
    df.insert(loc=0, column='month', value=j)
    # 插入收益率列
    df.insert(loc=0, column='yield_rate', value=np.nan)
    # 插入行业类别列
    df.insert(loc=0, column='sector_code', value=np.nan)
    # 插入板块类别列
    df.insert(loc=0, column='board_type', value=np.nan)

    # 遍历当前截面的每只股票
    for i in df.index.values.tolist():

        # 获取行业和板块信息
        df.loc[i, 'sector_code'] = instruments(order_book_ids=i).sector_code
        df.loc[i, 'board_type'] = instruments(order_book_ids=i).board_type

        # 获取当前日期和未来日期
        day = trading_dates_list_interval[df.loc[i, 'month']]
        day_interval = get_next_trading_date(date=day, n=para.yield_rate_interval)

        # 获取当前日期和未来日期停牌状态
        day_is_suspended = bool(is_suspended(order_book_ids=i, start_date=day, end_date=day).values[0][0])
        day_interval_is_suspended = bool(is_suspended(order_book_ids=i, start_date=day, end_date=day).values[0][0])

        # 判断是否停牌
        if day_is_suspended == False and day_interval_is_suspended == False:

            # 获取当前日期和未来日期的价格
            day_price = get_price(order_book_ids=i, start_date=day, end_date=day, fields='close')
            day_interval_price = get_price(order_book_ids=i, start_date=day_interval, end_date=day_interval, fields='close')

            # 判断是否数据缺失
            if day_price is None or day_interval_price is None:
                print("day_price or day_interval_price is None: {}".format(i))
            else:
                # 判断距离上市天数
                listed_date = datetime.datetime.strptime(instruments(order_book_ids=i).listed_date, '%Y-%m-%d').date()
                how_long_listed = (trading_dates_list_interval[j]-listed_date).days

                if how_long_listed > 90:
                    yield_rate = day_interval_price['close'].item() / day_price['close'].item() - 1
                    df.loc[i, 'yield_rate'] = yield_rate
                else:
                    print('how_long_listed <= 90d: {}'.format(i))

        else:
            print("day_is_suspended or day_interval_is_suspended == True: {}".format(i))

    # 写入文件
    df.to_csv(para.write_folder_path + '/' + str(j) + '.csv')

print()
print('Done!')


number of trading_dates_list_interval: 0/2410
use time: 0.000997781753540039

number of trading_dates_list_interval: 1/2410
use time: 1.8062567710876465

number of trading_dates_list_interval: 2/2410
use time: 2.160863161087036

number of trading_dates_list_interval: 3/2410
use time: 2.4945521354675293

number of trading_dates_list_interval: 4/2410
use time: 2.8445942401885986

number of trading_dates_list_interval: 5/2410
use time: 3.185401439666748

number of trading_dates_list_interval: 6/2410
use time: 3.53851056098938

number of trading_dates_list_interval: 7/2410
use time: 3.925309896469116

number of trading_dates_list_interval: 8/2410
use time: 4.278735876083374

number of trading_dates_list_interval: 9/2410
use time: 4.634817600250244

number of trading_dates_list_interval: 10/2410
use time: 4.980140447616577

number of trading_dates_list_interval: 11/2410
use time: 5.349115610122681

number of trading_dates_list_interval: 12/2410
use time: 5.688831806182861

number of tradin